# CIA Factbook Project Using SQL

### Introduction

For this project, we shake things up a little bit and shift gears to using SQL, or Structured Query Language. SQL is a fundamental tool in any Data Scientist's tool belt, and as a result, I would like to showcase my abilities by performing analysis using this quintessential language

For this project, we will be looking at a CIA World Factbook data set that contains a plethora of information and statistics about countries across the globe. The rest of this project will be dedicated to extracting and interpreting data to garner insights.

### Preliminary Procedures

We first begin by connecting to our database to retrieve our information for this project.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Let's get a better understanding of what our data from the database above looks like:

In [5]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Summary Statistics

#### Max and Min

This appears to be a fairly straightforward table. Nothing too crazy to see here. However, there are a lot of numbers, which is fantastic if we want to perform some analysis, which of course we do. For our first query, let's return the minimum and maximum of our population and our population growth columns respectively.

In [10]:
%%sql
SELECT MIN(population) AS 'Population Min', 
MAX(population) AS 'Population Max', 
MIN(population_growth) AS 'Growth Min', 
MAX(population_growth) AS 'Growth Max'
  FROM facts;

Done.


Population Min,Population Max,Growth Min,Growth Max
0,7256490011,0.0,4.02


It appears that there are invalid entries in some of our columns, particularly the ones relating to population. One country has a population of 0 while the other has a population of over 7.2 billion. Both cases are impossible. We need to drill down a bit deeper. 

In [13]:
%%sql 
SELECT name, population
  FROM facts
  ORDER BY population
  LIMIT 20;

Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None
Bouvet Island,None
Jan Mayen,None
British Indian Ocean Territory,None
South Georgia and South Sandwich Islands,None
Navassa Island,None


It would appear that a number of countries do not have information available to return. However, Antarctica was the country giving us problems. That's good to know. Let's see the maximum populations for countries.

In [14]:
%%sql
SELECT name, population
  FROM facts
 ORDER BY population DESC
 LIMIT 10;

Done.


name,population
World,7256490011
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745


Our table considers the world as a country. This makes a lot of sense. We will need to consider this going forward. let's try running the query from above, but this time exclude the world and antarctica.

In [25]:
%%sql
SELECT MIN(population) AS 'Population Min', 
MAX(population) AS 'Population Max', 
MIN(population_growth) AS 'Growth Min', 
MAX(population_growth) AS 'Growth Max'
  FROM facts
 WHERE name <> "World" AND name <> "Antarctica";

Done.


Population Min,Population Max,Growth Min,Growth Max
48,1367485388,0.0,4.02


Interesting! From this querey, we can see that the country with the least amount of people only has 48 while the country with the most people has about 1.4 billion. That's a lot!

#### Averages

Let's compute the averages for our population and area columns, excluding the world and antarctica.

In [26]:
%%sql
SELECT AVG(population) AS "Average Population", AVG(area) AS "Average Area"
  FROM facts
 WHERE name <> "World" AND name <> "Antarctica";

Done.


Average Population,Average Area
32377011.0125,555093.546184739


Interesting. It appears that the average population for a country is in the neighborhood of about 32 million while the average area of a country is 555,000 square miles. 

Let's explore this notion of average population further. A good question to answer is, "Which countries are above average for population but below average for land area?"

In [32]:
%%sql
SELECT name, population, area
  FROM facts
 WHERE population > (SELECT AVG(population) FROM facts) AND area < (SELECT AVG(area) FROM facts);
 ORDER BY 

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210
